In [1]:
!pip install selenium
!pip install selenium-wire


In [2]:
import sys

print(sys.version)

3.9.7 (default, Sep 16 2021, 16:59:28) [MSC v.1916 64 bit (AMD64)]


In [3]:
from json.decoder import JSONDecodeError

In [4]:
from tqdm import tqdm
from seleniumwire import webdriver 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from time import sleep
import requests
import json
import pandas as pd


#[출처] 구글맵 리뷰 크롤러 만들기|작성자 파플
#https://blog.naver.com/1012rnjsdydgns/222372764533

def google_reviews(store_list,gu,count=10):

    #드라이버 실행
    options = webdriver.ChromeOptions()
    # options.add_argument("Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36")
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome('C:/Users/user/Desktop/Final_Project/Data_crawling/selenium/chromedriver.exe',options=options)   #chromedriver 경로 지정!
    driver.get('https://www.google.co.kr/maps')
    sleep(15)

    # 리스트 형식으로 저장된 가게 하나씩 검색
    for store in store_list:
        count=count
        driver.get('https://www.google.co.kr/maps')
        result_list=[]
        sleep(3)
        query_input=driver.find_element_by_class_name('tactile-searchbox-input')
        query_input.send_keys(gu+store+'\n')
        
        # 의도한 가게와 검색결과가 같은지 확인
        sleep(5)
        check=input('일치하는 가게를 클릭 후 y를 눌러주세요(이외 멈춤)')
        if check.upper()=='Y':
            now_url=driver.current_url
        else:
            break
        
        # 리뷰 더보기로 이동
#         더보기 = driver.find_element_by_css_selector("button[aria-label*='리뷰 더보기']")
#         더보기.send_keys(Keys.ENTER)
        more_btn=driver.find_element_by_xpath('//*[@id="pane"]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]/span[1]/span/span/span[2]/span[1]/button')
        more_btn.click()
        views=int(''.join(more_btn.text[2:-1].split(',')))//10
        
        # div태그 스크롤 
        sleep(8)
        js_scripts = '''
        let aa = document.getElementsByClassName('section-scrollbox')[0];
        setTimeout(()=>{aa.scroll(0,1000000)}, 1000);
        '''
        driver.execute_script(js_scripts)
        sleep(3) 
        
        for i in range(5000):
            reviews = driver.find_elements(By.CSS_SELECTOR, value='span.section-review-text')
            stars = driver.find_elements(By.CSS_SELECTOR, value='span.section-review-stars')
            print(reviews)
            result_list.append({'내용':reviews,'별점':stars})        


        # csv로 저장
        df=pd.DataFrame(result_list)
        df.to_csv('{}.csv'.format(store))

In [5]:
water_list = ['캐리비안베이']

google_reviews(water_list,'한국',500)

WebDriverException: Message: 'chromedriver.exe' executable needs to be in PATH. Please see https://chromedriver.chromium.org/home
